<a href="https://colab.research.google.com/github/LuAsm/simple_python_project/blob/master/Copy_of_titanic_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.5 MB/s eta 0:00:00


In [1]:
import torch
import pytorch_lightning as pl
import pandas as pd
import requests
import tempfile

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from typing import List
import wandb

from pytorch_lightning.loggers import WandbLogger
from sklearn.linear_model import LogisticRegression

from time import time

In [2]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lukas-asmontas. Use `wandb login --relogin` to force relogin


True

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Task 1
Write a `Dataset` for Titanic Survivors data. Use columns `sex`, `age`, `sibsp` and `parch`. Apply data transformations where needed. Fill the missing values if there are any. 

Split the dataset into train, val, and test parts.

In [4]:
class TitanicDataset(Dataset):
  def __init__(self, df: pd.DataFrame, feature_names: List[str]):
    self._feature_names = feature_names
    self._scaler = StandardScaler()
    self._imputer = SimpleImputer(strategy="mean")

    self.data = self._process_data(df)
    
  def _process_data(self, df):
    df = df[["Survived", "Sex", "Age", "SibSp", "Parch"]]
    df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))

    self._imputer.fit(df[self._feature_names].values)
    self._scaler.fit(df[self._feature_names].values)

    return df

  def __len__(self):
    return self.data.shape[0]
    
  def __getitem__(self, idx):
    x = self.data.iloc[idx][self._feature_names].values
    y = self.data.iloc[idx]["Survived"]
    x_imputed = self._imputer.transform(x.reshape(1, -1))
    x_scaled = self._scaler.transform(x_imputed)[0]
    return x_scaled, y

In [5]:
train_df, test_df = train_test_split(data, train_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_df, train_size=0.5, random_state=42)

In [6]:
train_dataset = TitanicDataset(train_df, ["Age", "SibSp", "Parch", "Sex"])
test_dataset = TitanicDataset(test_df, ["Age", "SibSp", "Parch", "Sex"])
val_dataset = TitanicDataset(val_df, ["Age", "SibSp", "Parch", "Sex"])

<ipython-input-4-1566c9f0a086>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))
<ipython-input-4-1566c9f0a086>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))
<ipython-input-4-1566c9f0a086>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

### Task 1.1

Create dataloaders for each train, val, and test datasets. Select `batch_size`

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Task 2
Write a class for a network that can classify if the person surviced Titanic catastrophy. For now use only one linear layer and try out both [BCE](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) and [BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html). Connect WandB logger to track the loss. 

Train and evalute the model.

In [27]:
class Network(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = torch.nn.Linear(4, 1, dtype=torch.float64)
    torch.nn.init.kaiming_normal_(self.l1.weight, mode='fan_in', nonlinearity='sigmoid')
    self.bn = torch.nn.BatchNorm1d(num_features=1, dtype=torch.float64)

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(logits.view(-1), y)
    self.log("train_loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(logits.view(-1), y)
    self.log("val_loss", loss)

  def forward(self, x):
    x = self.l1(x)
    x = self.bn(x)
   # x = self.act(x)
   # x = self.l2(x)
    return x

In [23]:
  run_name = f"mlp_{time()}"
  wandb.init(name=run_name, project="codeacademy_titanic")

In [24]:
wandb_logger = WandbLogger()

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [28]:
model = Network()

In [29]:
trainer = pl.Trainer(max_epochs=50, logger=wandb_logger, log_every_n_steps=5)
trainer.fit(model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type        | Params
-------------------------------------
0 | l1   | Linear      | 5     
1 | bn   | BatchNorm1d | 2     
-------------------------------------
7         Trainable params
0         Non-trainable params
7         Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [19]:
# Inference
y_true = []
y_pred = []
for batch in test_dataloader:
  x, y = batch
  y_true.extend(y.tolist())
  logits = model.forward(x)
  probs = torch.sigmoid(logits)
  y_pred.extend((probs > 0.5).int().view(-1).tolist())

In [20]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.86      0.83       212
         1.0       0.77      0.67      0.72       144

    accuracy                           0.79       356
   macro avg       0.78      0.77      0.77       356
weighted avg       0.79      0.79      0.78       356



### Task 2.1
Save test results in the same WandB run. [Hint](https://github.com/dqmis/ntpps/blob/3c8b8d242eef3a84983cb634b14bc47ad0b2911f/scripts/train.py#L45)

In [22]:
from pytorch_lightning.callbacks import ModelCheckpoint
from datetime import datetime

In [41]:
import re
from sklearn.metrics import accuracy_score, classification_report, f1_score


In [52]:
wandb.finish()

In [21]:
report= pd.DataFrame(
            classification_report(y_true, y_pred, output_dict=True)
        ).transpose()

wandb.log({"classification report":report})
#wandb.log({"f1 score":report})
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,▆▇▅▅▆▄▆▄▆▄▆▆▄▅▃▄▅▁▃▆▄▆█▃▃▅▄▅▆▄▄▆▄▂▅▅▂▂▂▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▇▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch,49
train_loss,0.53474
trainer/global_step,299
val_loss,0.51931


## Task 3
Try initializing model weights using [Kaiming](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_normal_) initialization. Compare how does the train and val loss differ from a default initialization. [HINT](https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch)

## Task 4
Add Batch Norm layer after a Linear layer. Compare train and val loss changes to the default and Kaiming initiailization ones.